In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
# import sys
# sys.path.insert(1, )

from preprocessing import market_prepro

Now that we have Spencer's data importer lets make a driver that pulls in the data and training the model

SUDO CODE:

1. call data importer

2. restructure the dataset based on number of days desired [function written]

3. train model with training data

4. test model

5. return metrics

In [10]:
# st = "Stocks"
st = "ETFs"

#Input stock name
sn = "aadr" 
f = r'D:\Desktop\College Spring 2023\machineLearning\project\coding\data'
X_train, X_test, T_train, T_test = market_prepro(f,st,sn)

print(T_train)


Date
2010-07-21    23.946
2010-07-22    24.487
2010-07-23    24.507
2010-07-26    24.595
2010-07-27    24.517
               ...  
2015-09-08    36.193
2015-09-09    35.967
2015-09-10    36.223
2015-09-11    35.938
2015-09-14    35.938
Name: Close, Length: 1044, dtype: float64


In [9]:
def structure_timeseries_features(df,offset,exclude):
    '''
    This takes a dataframe and creates new columns that contain the data from
    previous days so that time series forecasting can occur.

    INPUTS:
        df - dataframe: input data

        offset - int: the number of days to go back. This creates this many
                        new columns

        exclude - list str: list of columns to exclude from the time series 
                            expansion

    OUTPUTS:
        df_out - dataframe: dataframe with new columns
    '''

    df_out = pd.DataFrame() 

    for cc in df.columns.values:
        if (cc not in exclude):
            for ii in range(offset):
                col_name = (cc+"_m"+str(ii+1))

                df_out[col_name] = df[cc].shift(ii+1)


    df_out = df_out.iloc[offset:,:]

    return df_out

def structure_timeseries_targets(df,offset):
    '''
    companion function to remove the first few days of targets to make sure
    the sizes match between features and targets.

    INPUTS:
        df - dataframe: input data targets.

    OUTPUTS:
        df_out - dataframe: dataframe with rows removed

    '''
    df_out = df.iloc[offset:,:]

    return df_out

# df_ts = structure_timeseries_features(tt,2, ['Day_date','Month','Year'])


In [ ]:
def modeldriver(mdl, n_days, f, st, sn):
    '''
    This function takes a model, imports the data, trains the model
    '''